# Tutorial: Distance Restraints

We can apply a harmonic restraint to atoms/residues if we are interested in changes to molecular interactions or thermodynamics.

This is another simplified tutorial building on what we learned from simulations of alanine dipeptide. We will simulate 2 molecules of alanine dipeptide, gradualy pull them away from each other.

We will perform the typical simulations steps, such as equilibration (minimizatino, heating 20 ps) and production runs (100 ps). Additionally, we will prepare 4 copies of this run, adjusting the distance restraint value.

The files we need are:

1. Topology file (*.parm7)
2. Cooridnate file (*.rst7)
3. MD input files (min.in, heat.in, prod.in, and *cv.rst*)

Activated your `ambertools` conda environment.

```bash
conda activate ambertools
```

## Preparing Inputs

Make a new working directory.

```bash
mkdir restraints
cd restraints
```

Since alanine dipeptide is an amino acid that is capped. We can use `tleap` to generate our topology and coordinate files. 

We can use tleap to generate the topology and coordinate file

To do so, make a file called [tleap.in](example/distance/tleap.in)

```bash
source leaprc.protein.ff19SB
source leaprc.water.TIP3P

sys1 = sequence { ACE ALA NME }
sys2 = sequence { ACE ALA NME }
translate sys2 { 5 0 0 }
system = combine { sys1 sys2 }

solvatebox system TIP3PBOX 12.0 iso 0.8

saveamberparm system step3_pbcsetup.parm7 step3_pbcsetup.rst7
savepdb system step3_pbcsetup.pdb

quit
```

Here we made each molecule of alanine dipeptide as `sys1` or `sys`. `tleap` will automatically place both in the center of the water box. We can change this by using `translate` to move one of the molecules 5 Å away in the x-direction. Finally, combine both molecules, and save a new PDB file. The PDB file will help with identifing atom/residue indexes. Get the Amber input files by running the script, `tleap.in`

```bash
tleap -sf tleap.in
```

If you list the files in the current working directory, you should see

1. [step3_pbcsetup.parm7](example/distance/step3_pbcsetup.parm7)
2. [step3_pbcsetup.rst7](example/distance/step3_pbcsetup.rst7)

*ALWAYS VISUALIZE THESE TWO FILES BEFORE MOVING ON!*

Next, we will make the Amber MD input files. 

They contain settings for each MD run. We will have 3 steps:

1. Minimization
2. Heating (for 10 ps at 300 K)
3. Production MD (10 ps at 300 K, ranging distance)

### Minimization

Make a [min.in](example/distance/min.in) file, with:

```bash
Minimize
 &cntrl
  imin=1    
  ntx=1,        
  irest=0,
  maxcyc=2000,
  ncyc=1000,
  ntpr=100,
  ntwx=0,
  cut=8.0,
 /
```

### Heating 

Make a [heat.in](example/distance/heat.in) file, with:

```bash
A NVT simulation for common production-level simulations
 &cntrl
    imin=0,        ! No minimization
    irest=0,       ! This is NOT a restart of an old MD simulation
    ntx=1,         ! So our inpcrd file has no velocities

    ! Temperature control
    ntt=3,         ! Langevin dynamics
    gamma_ln=1.0,  ! Friction coefficient (ps^-1)
    temp0=300,     ! Target temperature

    ! Potential energy control
    cut=9.0,       ! nonbonded cutoff, in angstroms

    ! MD settings
    nstlim=10000,  ! 10K steps, 10 ps total
    dt=0.001,      ! time step (ps)

    ! SHAKE
    ntc=2,         ! Constrain bonds containing hydrogen
    ntf=2,         ! Do not calculate forces of bonds containing hydrogen

    ! Control how often information is printed
    ntpr=1000,     ! Print energies every 1000 steps
    ntwx=1000,     ! Print coordinates every 5000 steps to the trajectory
    ntwr=1000,     ! Print a restart file every 10K steps (can be less frequent)
    ntxo=2,        ! Write NetCDF format
    ioutfm=1,      ! Write NetCDF format (always do this!)

    ! Wrap coordinates when printing them to the same unit cell
    iwrap=1,

    ! Restraint options
    ntr=1,
    restraint_wt=5,
    restraintmask=':1-6', ! Apply weak positional restraint on residues
 /
```

### Production MD

Make 4 copies of `prod.in`. Make sure they have different names (i.e. prod0, prod1, prod2, and prod3). Use the following as your template, and be sure to change the last line, where `DUMPAVE=prod.cv` to `DUMPAVE=prod[0,1,2, or 3].cv`.

```bash
A NVT simulation for common production-level simulations
 &cntrl
    imin=0,        ! No minimization
    irest=1,       ! This IS a restart of an old MD simulation
    ntx=5,         ! So our inpcrd file has velocities

    ! Temperature control
    ntt=3,         ! Langevin dynamics
    gamma_ln=1.0,  ! Friction coefficient (ps^-1)
    temp0=300,   ! Target temperature

    ! Potential energy control
    cut=9.0,       ! nonbonded cutoff, in Angstroms

    ! MD settings
    nstlim=10000, ! 2 ns total
    dt=0.001,      ! time step (ps)

    ! SHAKE
    ntc=2,         ! Constrain bonds containing hydrogen
    ntf=2,         ! Do not calculate forces of bonds containing hydrogen

    ! Control how often information is printed
    ntpr=1000,     ! Print energies every 1000 steps
    ntwx=1000,    ! Print coordinates every 50000 steps to the trajectory
    ntwr=1000,    ! Print a restart file every 10K steps (can be less frequent)
    ntxo=2,        ! Write NetCDF format
    ioutfm=1,      ! Write NetCDF format (always do this!)

    ! Wrap coordinates when printing them to the same unit cell
    iwrap=1,
        
    ! Restraints
    nmropt=1,      ! Turn on restraints

 /
 
 &wt type='DUMPFREQ', istep1=10000 /
 &wt type='END' /
 DISANG=cv.rst
 DUMPAVE=prod.cv

 /
```

Check if your files match these: [prod0.in](example/distance/prod0.in), [prod1.in](example/distance/prod1.in), [prod2.in](example/distance/prod2.in), and [prod3.in](example/distance/prod3.in)


### Restraint File

Like the `prod.in`, prepare 4 copies of the restraint file, `cv.rst`. Range the restraint value at 4, 6, 8, and 10. The file has the following format:

`iresid` - Residue numbers of 2 residues we apply the restraint to.
`r1..r4` - The shape of the potential.
`rk2..rk3` - Restraint force constant (kcal/mol)

```bash
# Distance restraint between residue 2 and 5 (the ALA part)
 &rst
  iresid=2,5
  r1=-10, r2=4, r3=4, r4=10,
  rk2=10.0, rk3=10.0,
 &end
```
 
Check if your files match these: [cv0.rst](example/distance/cv0.rst), [cv1.rst](example/distance/cv1.rst), [cv2.rst](example/distance/cv2.rst), and [cv3.rst](example/distance/cv3.rst)

## Run Amber MD with `sander`

You can prepare a single file running the steps sequentially. I made the file, [runmd.sh](example/distance/runmd.sh).

```bash
#!/bin/bash

sander -O -i min.in -p step3_pbcsetup.parm7 -c step3_pbcsetup.rst7 -o min.out -r min.ncrst -inf min.mdinfo

sander -O -i heat.in -p step3_pbcsetup.parm7 -c min.ncrst -o heat.out -r heat.ncrst -inf heat.mdinfo -ref min.ncrst -x heat.nc

sander -O -i prod0.in -p step3_pbcsetup.parm7 -c heat.ncrst -o prod0.out -r prod0.ncrst -inf prod0.mdinfo -x prod0.nc

sander -O -i prod1.in -p step3_pbcsetup.parm7 -c prod0.ncrst -o prod1.out -r prod1.ncrst -inf prod1.mdinfo -x prod1.nc

sander -O -i prod2.in -p step3_pbcsetup.parm7 -c prod1.ncrst -o prod2.out -r prod2.ncrst -inf prod2.mdinfo -x prod2.nc

sander -O -i prod3.in -p step3_pbcsetup.parm7 -c prod2.ncrst -o prod3.out -r prod3.ncrst -inf prod3.mdinfo -x prod3.nc

```

## Conclusion

Now you can visual the results with VMD or Chimera, and perform some analysis.
